In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [ ]:
all_schools_url = 'https://www.sports-reference.com/cbb/seasons/men/2023-school-stats.html'
response = requests.get(all_schools_url, timeout = 10)
print(response.status_code)

team_name_id_dict = {}

soup = BeautifulSoup(response.content)
table = soup.findAll('tr')

for row in table:
    
    search = row.find('a',href = True)
    if search:
        url_suffix = str(search).split('"')[1].replace(".html","")
        team_name = str(search).split(">")[1].replace("</a","").strip()
        team_name_id_dict[team_name] = url_suffix

print(team_name_id_dict['Purdue'])

In [ ]:
master_df = pd.DataFrame()
counter = 0
stop_to_rest_point = np.random.randint(100,300)

url_prefix = 'https://www.sports-reference.com'

for team_name,url_suffix in tqdm(team_name_id_dict.items()):

    full_url = f"{url_prefix}{url_suffix}-gamelogs.html"
    temp_df = pd.read_html(full_url)[0]
    temp_df.columns = [col2 if (col1.startswith('Unnamed') or col1 == "School") else f"opp_{col2}" for col1,col2 in temp_df.columns]
    temp_df = temp_df.iloc[:,~temp_df.columns.str.startswith('Unnamed')].drop('G',axis = 1).dropna().query("Date != 'Date'")
    temp_df['team_name'] = team_name
    master_df = pd.concat([master_df,temp_df])
    counter +=1
    if counter == stop_to_rest_point:
        time.sleep(np.random.randint(60,120))
        continue
  
    time.sleep(np.random.rantint(3,7))

In [ ]:
master_df